In [1]:
import pandas as pd 
import os
pen_info= pd.concat([pd.read_csv(os.path.join('/home/rajesh/work/failed_milkings/events_data',file)) for file in os.listdir('/home/rajesh/work/failed_milkings/events_data') if file.startswith('events')])
failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')
pen_info= pen_info[pen_info['PEN'].isin([30,31,32,33,34,35,36])]
pen_info.PEN.value_counts()
pen_info['Date']=pd.to_datetime(pen_info['Date'], format='mixed')

pen_info['Animal Number']=pen_info['ID'].astype(int)

failed_df['Animal Number']= failed_df['Animal Number'].astype(int)

failed_df['Date Time']= pd.to_datetime(failed_df['Date Time'])

failed_df = failed_df.reset_index(drop=True)
pen_info = pen_info.reset_index(drop=True)


import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

# --- Ensure clean datetime and consistent Animal Number type ---
failed_df['Date Time'] = pd.to_datetime(failed_df['Date Time'])
pen_info['Date'] = pd.to_datetime(pen_info['Date'])
failed_df['Animal Number'] = failed_df['Animal Number'].astype(str)
pen_info['Animal Number'] = pen_info['Animal Number'].astype(str)

# --- Create a lookup dict for pen_info grouped by Animal Number ---
pen_groups = {k: v for k, v in pen_info.groupby('Animal Number')}

# --- Function to find the nearest match for one row ---
def find_nearest(row):
    animal = row['Animal Number']
    event_time = row['Date Time']
    
    # If animal not in pen_info, return row unchanged (with NaNs for pen columns)
    if animal not in pen_groups:
        return {**row.to_dict(), 'Matched Date': pd.NaT}
    
    df_animal = pen_groups[animal]
    # Find index of nearest date
    nearest_idx = (df_animal['Date'] - event_time).abs().idxmin()
    nearest_row = df_animal.loc[nearest_idx].to_dict()
    
    # Merge dictionaries
    merged = {**row.to_dict(), **nearest_row}
    merged['Matched Date'] = nearest_row['Date']
    return merged

# --- Run in parallel across all CPU cores ---
num_cores = max(cpu_count() - 1, 1)  # leave one core free
with Pool(num_cores) as pool:
    merged_records = pool.map(find_nearest, [row for _, row in failed_df.iterrows()])

# --- Combine into final DataFrame ---
merged_df = pd.DataFrame(merged_records)

print("✅ Parallel nearest-date merge completed successfully!")
print(merged_df.head())



/tmp/ipykernel_2766/3707399845.py:4: DtypeWarning: Columns (6,49,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')


✅ Parallel nearest-date merge completed successfully!
   Unnamed: 0.2  Unnamed: 0.1 Animal Number  Unnamed: 1  Unnamed: 0  \
0             0             1             3           1           1   
1             1             2             3           2           2   
2             2             3             3           3           3   
3             3             4             3           4           4   
4             4             5             3           5           5   

   Animal Number.1 Gestation  Animal Name  Animal Tag Id  Conductivity LR  \
0                3         3          NaN      6697303.0             66.0   
1                3         3          NaN      6697303.0             65.0   
2                3         3          NaN      6697303.0             67.0   
3                3         3          NaN      6697303.0             64.0   
4                3         3          NaN      6697303.0             63.0   

   ...   DIM       Date            Remark   R  T   B    

In [10]:
merged_df.groupby(['Device Name', 'PEN']).size().reset_index(name='Count').sort_values(['Device Name', 'Count'], ascending=[True, False]).drop_duplicates('Device Name')


,Device Name,PEN,Count
0,Astronaut 101,30.0,24734
6,Astronaut 102,30.0,31616
12,Astronaut 103,30.0,29975
19,Astronaut 104,31.0,29319
25,Astronaut 105,31.0,28176
31,Astronaut 106,31.0,26098
38,Astronaut 107,32.0,29534
44,Astronaut 108,32.0,26463
50,Astronaut 109,32.0,25747
57,Astronaut 110,33.0,39238


In [19]:
merged_df_filtered= merged_df[merged_df['Device Name'].isin(
    ['Astronaut 107', 'Astronaut 109', 'Astronaut 108', 'Astronaut 106',
       'Astronaut 105', 'Astronaut 104', 'Astronaut 110', 'Astronaut 111',
       'Astronaut 112', 
       'Astronaut 103', 'Astronaut 102', 'Astronaut 101', 'Astronaut 118', 'Astronaut 117',
        'Astronaut 116',
       'Astronaut 113',
       'Astronaut 115', 
       'Astronaut 114'])
      ]

In [32]:
merged_df_filtered['correct_pen'] = merged_df_filtered['Device Name'].apply(
    lambda x: 30 if x.endswith(('101', '102', '103'))
    else 31 if x.endswith(('104', '105', '106'))
    else 32 if x.endswith(('107', '108', '109'))
    else 33 if x.endswith(('110', '111', '112'))
    else 34 if x.endswith(('113', '114', '115'))
    else 35
)


/tmp/ipykernel_2766/1935382931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['correct_pen'] = merged_df_filtered['Device Name'].apply(


In [34]:
merged_df_filtered.to_csv('/home/rajesh/work/failed_milkings/corrected_pen_df.csv')

In [1]:
import pandas as pd
merged_df_filtered= pd.read_csv('/home/rajesh/work/failed_milkings/corrected_pen_df.csv')

/tmp/ipykernel_46851/2537943519.py:2: DtypeWarning: Columns (7,50,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df_filtered= pd.read_csv('/home/rajesh/work/failed_milkings/corrected_pen_df.csv')


In [5]:
merged_df_filtered.sort_values(['Date Time'])['Date Time']

17582     2020-02-01 09:26:00
31120     2020-02-01 10:33:00
26327     2020-02-01 11:02:00
43721     2020-02-01 11:32:00
17583     2020-02-01 16:10:00
                 ...         
182645    2024-06-30 23:54:00
232098    2024-06-30 23:54:00
503631    2024-06-30 23:56:00
469245    2024-06-30 23:57:00
396681    2024-06-30 23:58:00
Name: Date Time, Length: 619734, dtype: object

In [6]:
merged_df_filtered.failed.value_counts()

failed
0.0    614681
1.0      5053
Name: count, dtype: int64

(1|anial_id)+ (pen/robot_id)

In [ ]:
(1|Cow) + (1|robot:Pen)+ pen , 
 mean + variance 


In [7]:
merged_df_filtered['Visit Result']= merged_df_filtered['Visit Result'].apply(lambda x: 'Other' if x in ['Failed', 'Teats not found', 'Robot arm stopped'] else x)
merged_df_filtered['visit_results_new']= merged_df_filtered['Visit Result'].apply(lambda x: 'Connection attempt' if x.startswith('Connection attemp') else ('Dead milk time' if x.startswith('Dead milk') else x) )

TypeError: 'int' object is not iterable

In [14]:
merged_df_filtered['visit_results_new'].value_counts().reset_index().rename(columns={'visit_results_new': 'Count'}).iloc[1:]['Count'].apply(lambda x: x / sum(x))


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [7]:
merged_df_filtered['weeks']= merged_df_filtered['Lactation Days'].astype(int).apply(lambda x: round(x/7)+1)
merged_df_filtered['seasons_milking']= pd.to_datetime(merged_df_filtered['Date Time']).dt.month.apply(lambda x: 'winter' if x in [12, 1, 2] else ('spring' if x in [3, 4, 5] else ('summer' if x in [6, 7, 8] else 'fall')))
merged_df_filtered['parity']= merged_df_filtered['Lactation Number'].astype(str).apply(lambda x: 'Primiparous' if x.strip()=='1' else 'Multiparous')
merged_df_filtered['gestation']= merged_df_filtered['Lactation Number'].astype(str)

# df_followup = (
#     merged_df_filtered
#     .groupby(['Animal Number', 'gestation'], as_index=False)
#     .agg(followup_weeks=('weeks', lambda x: x.max(skipna=True) - x.min(skipna=True) + 1))
# )

# merged_df_filtered = pd.merge(
#     merged_df_filtered,
#     df_followup,
#     on=['Animal Number', 'gestation'],
#     how='left'
# )


# Step 2: Aggregate binary failure merged_df_filtered into counts per group
agg_counts = (
    merged_df_filtered.groupby(['Animal Number','gestation', 'weeks', 'visit_results_new'], as_index=False)
        .agg(
            failed_count=('failed', 'sum'),
            )  # optional
        )


In [8]:
agg_counts= agg_counts[agg_counts['visit_results_new']!='Successful']

In [9]:
merged_df_filtered['Lactation Number'].value_counts()

Lactation Number
2    243291
1    159078
3    138910
4     62427
5     13399
6      2030
7       599
Name: count, dtype: int64

In [10]:
agg_counts['parity']= agg_counts['gestation'].astype(str).apply(lambda x: 'Primiparous' if x=='1' else 'Multiparous')

In [11]:
agg_counts['parity'].value_counts()

parity
Multiparous    2269
Primiparous    1129
Name: count, dtype: int64

In [12]:
agg_counts.failed_count.isna().sum()

0

In [80]:
agg_counts['Animal Number'].value_counts()

Animal Number
15355    50
18348    49
19766    46
18584    45
18026    41
         ..
19929     1
17905     1
17995     1
21321     1
3         1
Name: count, Length: 507, dtype: int64

In [13]:
agg_counts.to_csv('df_rate_ratios_without_device.csv')

In [56]:
weekly_frequency= merged_df_filtered.groupby([ 'Lactation Number', 'weeks', 'visit_results_new'])['failed'].sum().reset_index()
weekly_frequency['Lactation_Number']= weekly_frequency['Lactation Number'].apply(lambda x: 'First' if x==1 else 'Multiparous')

In [57]:
weekly_frequency

,Lactation Number,weeks,visit_results_new,failed,Lactation_Number
0,1,1,Connection attempt,1.0,First
1,1,1,Connection time,1.0,First
2,1,1,Successful,0.0,First
3,1,2,Connection attempt,12.0,First
4,1,2,Connection time,8.0,First
...,...,...,...,...,...
1215,7,30,Successful,0.0,Multiparous
1216,7,31,Successful,0.0,Multiparous
1217,7,32,Successful,0.0,Multiparous
1218,7,33,Successful,0.0,Multiparous
